In [8]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [114]:
%autoreload

from speem.instruments.power_supply.rudi_modules import RudiDAC, RudiHV, HVMode
import time

In [115]:
rudi = RudiHV(7)


In [122]:
rudi.set_setpoint(0)

Error while flushing the socket: flush_socket: maximum number of iterations reached


Error while flushing the socket: flush_socket: maximum number of iterations reached


ModbusError: Modbus Error: Exception code = 4

In [112]:
0<0

False

In [113]:
0>0

False